## Import Libraries

In [2]:
import os
from dotenv import load_dotenv
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever

c:\Users\LENOVO\anaconda3\envs\smartshopper\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

True

## Setup Document Stores

In [4]:
products_document_store = MongoDBAtlasDocumentStore(
    database_name="smartshopper_store",
    collection_name="products", 
    vector_search_index="vector_index",
    full_text_search_index="search_index",
)

common_info_document_store = MongoDBAtlasDocumentStore(
    database_name="smartshopper_store",
    collection_name="common_info",
    vector_search_index="common_info_vector_index", 
    full_text_search_index="common_info_search_index",
)

print(f"Products in store: {products_document_store.count_documents()}")
print(f"Common info in store: {common_info_document_store.count_documents()}")

Products in store: 5
Common info in store: 6


## Test Products Retrieval

### Create Products Retrieval Pipeline

In [5]:
embedder_768 = SentenceTransformersTextEmbedder()

In [6]:
products_pipeline = Pipeline()
products_pipeline.add_component("embedder", embedder_768)  # Use 768-dim embedder
products_pipeline.add_component("retriever", MongoDBAtlasEmbeddingRetriever(
    document_store=products_document_store, 
    top_k=5
))
products_pipeline.connect("embedder", "retriever")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: MongoDBAtlasEmbeddingRetriever
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])

### Test Product Queries


In [8]:
test_product_queries = [
    "smartphone with good camera",
    "comfortable running shoes", 
    "office chair for working",
    "cotton clothing",
    "wooden furniture"
]

print("\n=== PRODUCT RETRIEVAL TESTS ===")
for query in test_product_queries:
    print(f"\nQuery: '{query}'")
    result = products_pipeline.run({
        "embedder": {"text": query}
    })
    
    documents = result["retriever"]["documents"]
    print(f"Found {len(documents)} products:")
    
    for i, doc in enumerate(documents[:3], 1):  # Show top 3
        print(f"{i}. {doc.meta['title']} - {doc.meta['brand']}")
        print(f"   Price: Rp {doc.meta['price']:,}")
        print(f"   Material: {doc.meta['material']}, Category: {doc.meta['category']}")
        print(f"   Score: {doc.score:.3f}")


=== PRODUCT RETRIEVAL TESTS ===

Query: 'smartphone with good camera'


Batches: 100%|██████████| 1/1 [00:00<00:00, 11.26it/s]


Found 5 products:
1. Samsung Galaxy S24 - Samsung
   Price: Rp 15,000,000
   Material: Glass, Category: Electronics
   Score: 0.795
2. Cotton T-Shirt - EcoWear
   Price: Rp 250,000
   Material: Cotton, Category: Clothing
   Score: 0.563
3. Nike Air Max 270 - Nike
   Price: Rp 2,500,000
   Material: Synthetic, Category: Shoes
   Score: 0.555

Query: 'comfortable running shoes'


Batches: 100%|██████████| 1/1 [00:00<00:00, 18.13it/s]


Found 5 products:
1. Nike Air Max 270 - Nike
   Price: Rp 2,500,000
   Material: Synthetic, Category: Shoes
   Score: 0.863
2. Leather Office Chair - ErgoMax
   Price: Rp 3,500,000
   Material: Leather, Category: Furniture
   Score: 0.611
3. Cotton T-Shirt - EcoWear
   Price: Rp 250,000
   Material: Cotton, Category: Clothing
   Score: 0.580

Query: 'office chair for working'


...

Found 5 products:
1. Leather Office Chair - ErgoMax
   Price: Rp 3,500,000
   Material: Leather, Category: Furniture
   Score: 0.835
2. Wooden Dining Table - WoodCraft
   Price: Rp 8,000,000
   Material: Wood, Category: Furniture
   Score: 0.644
3. Nike Air Max 270 - Nike
   Price: Rp 2,500,000
   Material: Synthetic, Category: Shoes
   Score: 0.579

Query: 'cotton clothing'


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.54it/s]


Found 5 products:
1. Cotton T-Shirt - EcoWear
   Price: Rp 250,000
   Material: Cotton, Category: Clothing
   Score: 0.827
2. Leather Office Chair - ErgoMax
   Price: Rp 3,500,000
   Material: Leather, Category: Furniture
   Score: 0.655
3. Nike Air Max 270 - Nike
   Price: Rp 2,500,000
   Material: Synthetic, Category: Shoes
   Score: 0.618

Query: 'wooden furniture'


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.82it/s]


Found 5 products:
1. Wooden Dining Table - WoodCraft
   Price: Rp 8,000,000
   Material: Wood, Category: Furniture
   Score: 0.824
2. Leather Office Chair - ErgoMax
   Price: Rp 3,500,000
   Material: Leather, Category: Furniture
   Score: 0.706
3. Cotton T-Shirt - EcoWear
   Price: Rp 250,000
   Material: Cotton, Category: Clothing
   Score: 0.606


## Test Common Information Retrieval


### Create Common Info Retrieval Pipeline  


In [10]:
common_info_embedder = SentenceTransformersTextEmbedder()

common_info_pipeline = Pipeline()
common_info_pipeline.add_component("embedder", common_info_embedder)
common_info_pipeline.add_component("retriever", MongoDBAtlasEmbeddingRetriever(
    document_store=common_info_document_store,
    top_k=3
))
common_info_pipeline.connect("embedder", "retriever")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: MongoDBAtlasEmbeddingRetriever
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])

### Test Common Info Queries


In [25]:
test_info_queries = [
    "how to return items",
    "payment methods accepted",
    "shipping costs and delivery time",
    "refund process",
    "customer support hours"
]

print("\n=== COMMON INFORMATION RETRIEVAL TESTS ===")
for query in test_info_queries:
    print(f"\nQuery: '{query}'")
    result = common_info_pipeline.run({
        "embedder": {"text": query}
    })
    
    documents = result["retriever"]["documents"]
    print(f"Found {len(documents)} information pieces:")
    
    for i, doc in enumerate(documents, 1):
        print(f"{i}. {doc.meta['title']} (Category: {doc.meta['category']})")
        print(f"   Content: {doc.content[:100]}...")
        print(f"   Score: {doc.score:.3f}")


=== COMMON INFORMATION RETRIEVAL TESTS ===

Query: 'how to return items'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.84it/s]


Found 0 information pieces:

Query: 'payment methods accepted'


Batches: 100%|██████████| 1/1 [00:00<00:00, 16.45it/s]


...

Found 0 information pieces:

Query: 'refund process'


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.31it/s]


Found 0 information pieces:

Query: 'customer support hours'


Batches: 100%|██████████| 1/1 [00:00<00:00, 22.01it/s]


Found 0 information pieces:


## Test with Filters

In [28]:
print("\nTesting material filter (Cotton):")
material_filter = {
    "field": "material", 
    "operator": "==", 
    "value": "Cotton"
}

result = products_pipeline.run({
    "embedder": {"text": "clothing"},
    "retriever": {"filters": material_filter}
})

documents = result["retriever"]["documents"]
print(f"Found {len(documents)} cotton products:")
for doc in documents:
    print(f"- {doc.meta['title']} ({doc.meta['material']})")

### Test Category Filter 
print("\nTesting category filter (Electronics)")

category_filter = {
    "field": "category",
    "operator": "==",
    "value": "Electronics"
}

result = products_pipeline.run({
    "embedder": {"text": "device"},
    "retriever": {"filters": category_filter}
})

documents = result["retriever"]["documents"]
print(f"Found {len(documents)} electronics:")
for doc in documents:
    print(f"- {doc.meta['title']} ({doc.meta['category']})")

### Test Price Range Filter 
print("\nTesting price range filter (Under 5,000,000):")

price_filter = {
    "field": "price",  
    "operator": "<=",
    "value": 5000000
}

result = products_pipeline.run({
    "embedder": {"text": "product"},
    "retriever": {"filters": price_filter}
})

documents = result["retriever"]["documents"]
print(f"Found {len(documents)} products under Rp 5,000,000:")
for doc in documents:
    print(f"- {doc.meta['title']} - Rp {doc.meta['price']:,}")

### Test Multiple Filters (Advanced)
print("\nTesting multiple filters (Cotton AND price <= 1,000,000):")

multiple_filters = {
    "operator": "AND",
    "conditions": [
        {"field": "material", "operator": "==", "value": "Cotton"},
        {"field": "price", "operator": "<=", "value": 1000000}
    ]
}

result = products_pipeline.run({
    "embedder": {"text": "clothing"},
    "retriever": {"filters": multiple_filters}
})

documents = result["retriever"]["documents"]
print(f"Found {len(documents)} cotton products under Rp 1,000,000:")
for doc in documents:
    print(f"- {doc.meta['title']} - {doc.meta['material']} - Rp {doc.meta['price']:,}")

print("\nfilter testing completed!")


Testing material filter (Cotton) - Fixed format:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 18.98it/s]


Found 0 cotton products:

Testing category filter (Electronics) - Fixed format:


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.04it/s]


...

Batches: 100%|██████████| 1/1 [00:00<00:00, 28.38it/s]

Found 0 products under Rp 5,000,000:



Testing multiple filters (Cotton AND price <= 1,000,000):


Batches: 100%|██████████| 1/1 [00:00<00:00, 17.45it/s]


Found 0 cotton products under Rp 1,000,000:

Fixed filter testing completed!
